In [3]:
using ITensors
using Distributions
using HDF5
using Random
using Plots
using Plots.PlotMeasures
using MLDataUtils, StatsBase
using DelimitedFiles
include("../SweepOptRegular.jl")

InspectOverlap (generic function with 2 methods)

In [4]:
ecg_dat = readdlm("../ECG200_TRAIN.txt")
X_train = ecg_dat[:, 2:end]
y_train = Int.(ecg_dat[:, 1])
remap = Dict(-1 => 0, 1 => 1)
y_train = [remap[label] for label in y_train];
y_train = reshape(y_train, length(y_train), 1);

In [5]:
ecg_dat_test = readdlm("../ECG200_TEST.txt")
X_test = ecg_dat_test[:, 2:end]
y_test = Int.(ecg_dat_test[:, 1])
y_test = [remap[label] for label in y_test]
y_test = reshape(y_test, length(y_test), 1);

In [6]:
struct RobustSigmoidTransform{T<:Real} <: AbstractDataTransform
    median::T
    iqr::T
    k::T
    positive::Bool

    function RobustSigmoidTransform(median::T, iqr::T, k::T, positive=true) where T<:Real
        new{T}(median, iqr, k, positive)
    end
end

function robust_sigmoid(x::Real, median::Real, iqr::Real, k::Real, positive::Bool)
    xhat = 1.0 / (1.0 + exp(-(x - median) / (iqr / k)))
    if !positive
        xhat = 2*xhat - 1
    end
    return xhat
end

function fitScaler(::Type{RobustSigmoidTransform}, X::Matrix; k::Real=1.35, positive::Bool=true)
    medianX = median(X)
    iqrX = iqr(X)
    return RobustSigmoidTransform(medianX, iqrX, k, positive)
end

function transformData(t::RobustSigmoidTransform, X::Matrix)
    return map(x -> robust_sigmoid(x, t.median, t.iqr, t.k, t.positive), X)
end

# New SigmoidTransform
struct SigmoidTransform <: AbstractDataTransform
    positive::Bool
end

function sigmoid(x::Real, positive::Bool)
    xhat = 1.0 / (1.0 + exp(-x))
    if !positive
        xhat = 2*xhat - 1
    end
    return xhat
end

function fitScaler(::Type{SigmoidTransform}, X::Matrix; positive::Bool=true)
    return SigmoidTransform(positive)
end

function transformData(t::SigmoidTransform, X::Matrix)
    return map(x -> sigmoid(x, t.positive), X)
end;

In [7]:
rst = fitScaler(RobustSigmoidTransform, X_train; positive=true)
X_train_normalised = transformData(rst, X_train)
X_test_normalised = transformData(rst, X_test);

In [8]:
W, sites = fitMPS(X_train_normalised, y_train; χ_max=15, nsweep=10, α=0.01, χ_init=2, random_state=42)

Initialising training states.


Parameters: 
Feature map: AngleEncoder
Initial bond dimension χ_init: 2
Step size α: 0.01
Maximum number of sweeps: 10
Maximum bond dimension χ_max: 15
Generating initial weight MPS with bond dimension χ = 2 using random seed 42.


Forward Sweep L -> R (1/10)

Backward Sweep R -> L (1/10)


Training loss after Sweep 1: 0.320677773782609 | Training Acc: 0.8
ΔC train after sweep 1: 0.17932222621739102
Forward Sweep L -> R (2/10)


...

Backward Sweep R -> L (9/10)


Training loss after Sweep 9: 0.33101289985346616 | Training Acc: 0.98
ΔC train after sweep 9: 0.0005702810170226669
Forward Sweep L -> R (10/10)


Backward Sweep R -> L (10/10)


Training loss after Sweep 10: 0.3305346636499325 | Training Acc: 0.98
ΔC train after sweep 10: 0.00047823620353365515


(MPS
[1] ((dim=2|id=45|"S=1/2,Site,n=1"), (dim=2|id=637|"decision"), (dim=4|id=356|"Link,l=1"))
[2] ((dim=2|id=382|"S=1/2,Site,n=2"), (dim=8|id=811|"Link,l=2"), (dim=4|id=356|"Link,l=1"))
[3] ((dim=2|id=724|"S=1/2,Site,n=3"), (dim=15|id=205|"Link,l=3"), (dim=8|id=811|"Link,l=2"))
[4] ((dim=2|id=935|"S=1/2,Site,n=4"), (dim=15|id=967|"Link,l=4"), (dim=15|id=205|"Link,l=3"))
[5] ((dim=2|id=505|"S=1/2,Site,n=5"), (dim=15|id=80|"Link,l=5"), (dim=15|id=967|"Link,l=4"))
[6] ((dim=2|id=599|"S=1/2,Site,n=6"), (dim=15|id=233|"Link,l=6"), (dim=15|id=80|"Link,l=5"))
[7] ((dim=2|id=165|"S=1/2,Site,n=7"), (dim=15|id=221|"Link,l=7"), (dim=15|id=233|"Link,l=6"))
[8] ((dim=2|id=628|"S=1/2,Site,n=8"), (dim=15|id=135|"Link,l=8"), (dim=15|id=221|"Link,l=7"))
[9] ((dim=2|id=673|"S=1/2,Site,n=9"), (dim=15|id=177|"Link,l=9"), (dim=15|id=135|"Link,l=8"))
[10] ((dim=2|id=800|"S=1/2,Site,n=10"), (dim=15|id=49|"Link,l=10"), (dim=15|id=177|"Link,l=9"))
[11] ((dim=2|id=81|"S=1/2,Site,n=11"), (dim=15|id=997|"Link,l

In [75]:
ScoreMPS(W, X_test_normalised, y_test)

Initialising testing states.


Test loss: 0.35429153863957297 | Test Accuracy: 0.92


2-element Vector{Float64}:
 0.35429153863957297
 0.92

In [76]:
mps_state_0 = deepcopy(W);

In [77]:
norm(mps_state_0)

1.0000000000000033

In [78]:
decision_idx = findindex(mps_state_0[1], "decision")

(dim=2|id=637|"decision")

In [79]:
decision_state_0 = onehot(decision_idx => (1))
mps_state_0[1] *= decision_state_0

ITensor ord=2 (dim=2|id=45|"S=1/2,Site,n=1") (dim=4|id=356|"Link,l=1")
NDTensors.Dense{Float64, Vector{Float64}}

In [80]:
normalize!(mps_state_0)

MPS
[1] ((dim=2|id=45|"S=1/2,Site,n=1"), (dim=4|id=356|"Link,l=1"))
[2] ((dim=2|id=382|"S=1/2,Site,n=2"), (dim=8|id=811|"Link,l=2"), (dim=4|id=356|"Link,l=1"))
[3] ((dim=2|id=724|"S=1/2,Site,n=3"), (dim=15|id=205|"Link,l=3"), (dim=8|id=811|"Link,l=2"))
[4] ((dim=2|id=935|"S=1/2,Site,n=4"), (dim=15|id=967|"Link,l=4"), (dim=15|id=205|"Link,l=3"))
[5] ((dim=2|id=505|"S=1/2,Site,n=5"), (dim=15|id=80|"Link,l=5"), (dim=15|id=967|"Link,l=4"))
[6] ((dim=2|id=599|"S=1/2,Site,n=6"), (dim=15|id=233|"Link,l=6"), (dim=15|id=80|"Link,l=5"))
[7] ((dim=2|id=165|"S=1/2,Site,n=7"), (dim=15|id=221|"Link,l=7"), (dim=15|id=233|"Link,l=6"))
[8] ((dim=2|id=628|"S=1/2,Site,n=8"), (dim=15|id=135|"Link,l=8"), (dim=15|id=221|"Link,l=7"))
[9] ((dim=2|id=673|"S=1/2,Site,n=9"), (dim=15|id=177|"Link,l=9"), (dim=15|id=135|"Link,l=8"))
[10] ((dim=2|id=800|"S=1/2,Site,n=10"), (dim=15|id=49|"Link,l=10"), (dim=15|id=177|"Link,l=9"))
[11] ((dim=2|id=81|"S=1/2,Site,n=11"), (dim=15|id=997|"Link,l=11"), (dim=15|id=49|"Link,l

In [81]:
norm(mps_state_0)

1.000000000000001

In [82]:
function entropy_von_neumann(psi::MPS, b::Int)
    s = siteinds(psi)
    orthogonalize!(psi, b) # change orthogonality center to site B
    #print(norm(psi))
    if b == 1
        _, S = svd(psi[b], (siteind(psi, b),))
    else
        _, S = svd(psi[b], (linkind(psi, b-1), s[b]))
    end
    SvN = 0.0
    for n in 1:ITensors.dim(S, 1)
        p = S[n, n]^2
        SvN -= p * log(p)
    end

    return SvN
end

entropy_von_neumann (generic function with 2 methods)

In [84]:
entropy_von_neumann(mps_state_0, 1)

0.11355204084697462

# Minimum Value Measurements

We will begin by getting the average variation in EE (at each site) after having measured each MPS site to be zero (minimum) in value. To do this, we begin by constructing a projector to project the MPS state into the subspace where a particular site is measured to have a large and negative value.

In [85]:
AngleEncoder(0.01)

2-element Vector{Float64}:
 0.9998766324816606
 0.015707317311820675

We will do an initial test with just the first site - that is, let's see what happens when we measure the first site in the MPS to be zero. 

In [86]:
zero_projector = AngleEncoder(0.01) * AngleEncoder(0.01)'
zero_projector_operator = op(zero_projector, sites[1])
@show zero_projector_operator

zero_projector_operator = ITensor ord=2
Dim 1: (dim=2|id=45|"S=1/2,Site,n=1")'
Dim 2: (dim=2|id=45|"S=1/2,Site,n=1")
NDTensors.Dense{Float64, Vector{Float64}}
 2×2
 0.9997532801828658    0.015705379539064146
 0.015705379539064146  0.00024671981713422146


ITensor ord=2 (dim=2|id=45|"S=1/2,Site,n=1")' (dim=2|id=45|"S=1/2,Site,n=1")
NDTensors.Dense{Float64, Vector{Float64}}

Now we measure the EE before making the measurement.

In [87]:
S_1_allSites = [entropy_von_neumann(mps_state_0, b, sites) for b=1:length(mps_state_0) if b != 1];

Now we get the EE after making the measurement at site 1.

In [89]:
new_site_1 = zero_projector_operator * mps_state_0[1]
noprime!(new_site_1)
mps_state_0[1] = new_site_1

ITensor ord=2 (dim=2|id=45|"S=1/2,Site,n=1") (dim=2|id=418|"Link,l=1")
NDTensors.Dense{Float64, Vector{Float64}}

In [93]:
normalize!(mps_state_0)

MPS
[1] ((dim=2|id=45|"S=1/2,Site,n=1"), (dim=2|id=726|"Link,l=1"))
[2] ((dim=2|id=382|"S=1/2,Site,n=2"), (dim=4|id=536|"Link,l=2"), (dim=2|id=726|"Link,l=1"))
[3] ((dim=2|id=724|"S=1/2,Site,n=3"), (dim=8|id=734|"Link,l=3"), (dim=4|id=536|"Link,l=2"))
[4] ((dim=2|id=935|"S=1/2,Site,n=4"), (dim=15|id=467|"Link,l=4"), (dim=8|id=734|"Link,l=3"))
[5] ((dim=2|id=505|"S=1/2,Site,n=5"), (dim=15|id=496|"Link,l=5"), (dim=15|id=467|"Link,l=4"))
[6] ((dim=2|id=599|"S=1/2,Site,n=6"), (dim=15|id=321|"Link,l=6"), (dim=15|id=496|"Link,l=5"))
[7] ((dim=2|id=165|"S=1/2,Site,n=7"), (dim=15|id=723|"Link,l=7"), (dim=15|id=321|"Link,l=6"))
[8] ((dim=2|id=628|"S=1/2,Site,n=8"), (dim=15|id=550|"Link,l=8"), (dim=15|id=723|"Link,l=7"))
[9] ((dim=2|id=673|"S=1/2,Site,n=9"), (dim=15|id=82|"Link,l=9"), (dim=15|id=550|"Link,l=8"))
[10] ((dim=2|id=800|"S=1/2,Site,n=10"), (dim=15|id=77|"Link,l=10"), (dim=15|id=82|"Link,l=9"))
[11] ((dim=2|id=81|"S=1/2,Site,n=11"), (dim=15|id=355|"Link,l=11"), (dim=15|id=77|"Link,l=1

In [94]:
entropy_von_neumann(mps_state_0, 1, sites)

NaN

In [43]:
S_1_prime_allSites = [entropy_von_neumann(mps_state_0, b, sites) for b=1:length(mps_state_0) if b != 1];

In [46]:
sites

96-element Vector{Index{Int64}}:
 (dim=2|id=45|"S=1/2,Site,n=1")
 (dim=2|id=382|"S=1/2,Site,n=2")
 (dim=2|id=724|"S=1/2,Site,n=3")
 (dim=2|id=935|"S=1/2,Site,n=4")
 (dim=2|id=505|"S=1/2,Site,n=5")
 (dim=2|id=599|"S=1/2,Site,n=6")
 (dim=2|id=165|"S=1/2,Site,n=7")
 (dim=2|id=628|"S=1/2,Site,n=8")
 (dim=2|id=673|"S=1/2,Site,n=9")
 (dim=2|id=800|"S=1/2,Site,n=10")
 ⋮
 (dim=2|id=7|"S=1/2,Site,n=88")
 (dim=2|id=908|"S=1/2,Site,n=89")
 (dim=2|id=62|"S=1/2,Site,n=90")
 (dim=2|id=850|"S=1/2,Site,n=91")
 (dim=2|id=261|"S=1/2,Site,n=92")
 (dim=2|id=577|"S=1/2,Site,n=93")
 (dim=2|id=9|"S=1/2,Site,n=94")
 (dim=2|id=638|"S=1/2,Site,n=95")
 (dim=2|id=928|"S=1/2,Site,n=96")

In [ ]:
entropy_von_neumann(mps_state_0, 2, sites)

In [45]:
S_1_allSites

95-element Vector{Float64}:
  0.28449688252764344
  0.42808174871655513
  0.5275786003021998
  0.6169657610016974
  0.7372901574764118
  0.8344508534890703
  0.9273506868415833
  1.0294656730796006
  1.080287038812791
  1.1360984539305718
  ⋮
  0.6168425118747356
  0.5961456945496805
  0.5476249044756291
  0.5047878524859928
  0.453410850862611
  0.3503986094009979
  0.2408502797520761
  0.08623353414858678
 -4.440892098500627e-16